In [160]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

In [468]:
df = pd.read_csv('all_task_environments.csv', header=0, skiprows=[1])

In [494]:
def get_avg_editdist(data):
    results = pd.DataFrame()
    dimensions = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]
        
    for row in range(0,len(df)):
        objects = list(df.at[row,'objects'].split(','))
        strong_k = list(df.at[row,'strong_k'].split(','))
        mid_k = list(df.at[row,'mid_k'].split(','))
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
        start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
        sequence = str(df.at[row,'sequence'])
        
        for k in np.arange(0.0,1.0,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.05,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else 1.0 for obj in objects}
                    
            for c in np.arange(1.0,2.0,0.1):
                c = round(c, 1)
                c1 = {obj: c if obj in df.at[row, 'containment'] else 1.0 for obj in objects}
                
                for dim in dimensions:                
                    # get average edit distance
                    edit_dist = get_average(objects, coordinates, start_coordinates, c1, k1, dim, sequence)
                    
                    params = 'c: ' + str(c) + ', k: ' + str(k_strong) + ',' + str(k_mid) + ', dim: ' + str(dim[1])
                    results.at[row,params] = edit_dist
                    
    return results

#TODO:
# fix problem with xz
# range for diff between strong/mid k?
# range for c and k in general?

In [533]:
%%timeit -n1 -r1
results_new = get_avg_editdist(df)

5min 24s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [534]:
results_new

,"c: 1.0, k: 0.0,0.05, dim: x","c: 1.0, k: 0.0,0.05, dim: y","c: 1.0, k: 0.0,0.05, dim: z","c: 1.0, k: 0.0,0.05, dim: xy","c: 1.0, k: 0.0,0.05, dim: xz","c: 1.0, k: 0.0,0.05, dim: yz","c: 1.0, k: 0.0,0.05, dim: xyz","c: 1.1, k: 0.0,0.05, dim: x","c: 1.1, k: 0.0,0.05, dim: y","c: 1.1, k: 0.0,0.05, dim: z",...,"c: 1.8, k: 0.9,0.95, dim: xz","c: 1.8, k: 0.9,0.95, dim: yz","c: 1.8, k: 0.9,0.95, dim: xyz","c: 1.9, k: 0.9,0.95, dim: x","c: 1.9, k: 0.9,0.95, dim: y","c: 1.9, k: 0.9,0.95, dim: z","c: 1.9, k: 0.9,0.95, dim: xy","c: 1.9, k: 0.9,0.95, dim: xz","c: 1.9, k: 0.9,0.95, dim: yz","c: 1.9, k: 0.9,0.95, dim: xyz"
0,2.060,3.820,3.530,2.380,0.9900,1.4500,1.0000,1.050,2.8000,3.4800,...,1.5300,1.7100,0.550,0.9800,3.3300,2.390,1.53,1.5700,1.7000,0.5000
1,1.850,3.470,3.440,1.700,1.6700,3.5000,1.6600,0.950,3.5800,3.5500,...,2.5000,2.5700,2.550,0.8700,2.6500,2.710,2.51,2.5500,2.6400,2.4800
2,2.040,3.470,3.560,1.010,0.9800,3.4300,1.1400,1.050,3.4800,3.4500,...,1.4000,2.4400,1.450,0.9300,2.2300,2.380,1.51,1.4600,2.4000,1.4500
3,2.780,2.000,2.000,1.720,1.6500,2.0000,1.6700,2.130,2.0000,2.0000,...,0.5200,1.2500,0.410,2.0000,1.3600,1.300,0.52,0.5100,1.2900,0.5500
4,1.010,2.000,2.000,1.570,1.0000,2.0000,1.0000,0.000,2.0000,1.0000,...,0.0000,1.4600,1.000,0.0000,1.4500,2.000,1.55,0.0000,1.5200,1.0000
5,3.460,2.550,4.000,2.820,2.0000,1.4700,2.0000,3.530,1.4700,4.0000,...,2.0000,4.0000,4.000,3.2800,2.0000,4.000,3.08,2.0000,4.0000,4.0000
6,3.090,2.810,3.270,4.000,3.1800,4.0000,4.0000,2.920,3.0400,2.8000,...,3.0200,2.0000,2.000,2.9400,2.0000,2.920,2.00,3.0400,2.0000,2.0000
7,3.000,4.000,2.930,4.000,3.1200,4.0000,4.0000,4.000,4.0000,2.9300,...,4.0000,4.0000,4.000,4.0000,4.0000,2.810,4.00,4.0000,4.0000,4.0000
8,0.500,0.920,1.210,0.000,0.5300,0.9600,0.0000,0.430,0.9400,1.3400,...,0.4400,0.9800,0.000,0.4800,0.9400,1.260,0.00,0.5400,1.0200,0.0000
9,0.500,1.480,0.460,1.000,0.4800,1.0000,1.0000,0.560,1.5000,0.4000,...,0.5200,1.0000,1.000,0.5100,1.4400,0.580,1.00,0.4500,1.0000,1.0000


In [526]:
def get_lowest_error(results):
    for col in list(results):
        results.loc['mean',col] = results[col].mean()
    lowest = min(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)]
    

In [535]:
get_lowest_error(results_new)

(2.4524999999999997, Index(['c: 1.5, k: 0.4,0.45, dim: xy'], dtype='object'))

In [515]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [528]:
def get_average(objects, coordinates, start_coordinates, c, k, dimension, sequence):

    edit_list = []

    for x in range(0,100):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    avg = np.mean(edit_list)
    return avg